In [1]:
import pickle

import pandas as pd
import numpy as np

In [2]:
models = ['lgb', 'xgb'] #, 'lstm'

In [3]:
train_parts = []
valid_parts = []
test_parts = []

for model in models:
    with open('{}_train_pred.pickle'.format(model), 'rb') as f:
        train_part = pickle.load(f)
        train_part = train_part.reshape(-1, 1)
        print(train_part.shape)
        train_parts.append(train_part)

X = np.concatenate(train_parts, axis=1)
with open('{}_valid.pickle'.format('lgb'), 'rb') as f:
    y = pickle.load(f)

(4777260, 1)
(3344082, 1)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 4777260 and the array at index 1 has size 3344082

In [21]:
X.shape

(238172, 2)

In [22]:
y.shape

(238172,)

In [23]:
import xgboost as xgb

In [58]:
clf = xgb.XGBRegressor()

In [59]:
clf.fit(
    X, 
    y, 
    eval_metric="rmse", 
    eval_set=[(X, y)], 
    verbose=True, 
    early_stopping_rounds=10,
)

[0]	validation_0-rmse:1.11676
Will train until validation_0-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:1.07863
[2]	validation_0-rmse:1.04674
[3]	validation_0-rmse:1.01997
[4]	validation_0-rmse:0.997463
[5]	validation_0-rmse:0.978816
[6]	validation_0-rmse:0.963356
[7]	validation_0-rmse:0.950503
[8]	validation_0-rmse:0.939838
[9]	validation_0-rmse:0.930867
[10]	validation_0-rmse:0.923577
[11]	validation_0-rmse:0.917455
[12]	validation_0-rmse:0.912475
[13]	validation_0-rmse:0.908272
[14]	validation_0-rmse:0.904801
[15]	validation_0-rmse:0.901951
[16]	validation_0-rmse:0.899621
[17]	validation_0-rmse:0.897615
[18]	validation_0-rmse:0.896003
[19]	validation_0-rmse:0.894676
[20]	validation_0-rmse:0.893469
[21]	validation_0-rmse:0.892426
[22]	validation_0-rmse:0.891601
[23]	validation_0-rmse:0.890724
[24]	validation_0-rmse:0.890065
[25]	validation_0-rmse:0.889542
[26]	validation_0-rmse:0.889067
[27]	validation_0-rmse:0.888639
[28]	validation_0-rmse:0.888227
[29]	validation_0-rms

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=True, subsample=1)

In [60]:
preds = clf.predict(X)

In [61]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse_sklearn(y_actual, y_predicted):
    return sqrt(mean_squared_error(y_actual, y_predicted))

In [62]:
rmse_sklearn(y, preds)

0.8745526123709251

In [63]:
train_parts = []

for model in models:
    with open('{}_test_pred.pickle'.format(model), 'rb') as f:
        train_part = pickle.load(f)
        train_part = train_part.reshape(-1, 1)
        print(train_part.shape)
        train_parts.append(train_part)
    
test_X = np.concatenate(train_parts, axis=1)

(214200, 1)
(214200, 1)


In [64]:
test_preds = clf.predict(test_X)

In [65]:
from module.data.read_data import *

test = test_file_processing().set_index('ID')
test.shape

(214200, 3)

In [66]:
import pickle

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": test_preds.clip(0, 20),
})

submission.to_csv('ensamble_submission.csv', index=False)
pickle.dump(test_preds, open('ensamble_test.pickle', 'wb'))